# Scrapping images and BMI from different sources

#### Scrapping fro "http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/" website which has facial images of celebrities with their BMI

In [15]:

import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlretrieve


def get_page_urls(url):
    image_page_link = []
    for i in range(1,50):                 ## Here range(1,50) describes the number of pages on this website which may increase in future if the data is updated we need to update this range likewise.
        print(f"Scrapping image page links of {i} page")
        url = urljoin(url,f"?page{i}")    ## To create a url of specific page
        try:
            response = requests.get(url)
        except Exception as e:
            print(f'There is problem while accesing the given url: {url} and error is: {e} on page number {i}')
        
        try:
            soup = BeautifulSoup(response.content,"html.parser")
            parsing_1 = soup.find_all('div',{'class':'content_body'})[0].find_all('div',{'class':'proton_tile'})
        except Exception as e:
            print(f'There is problem in parsing the link {url}')
        
        try:
            for element in parsing_1:
                pg_links = element.find_all('a')
                for pg_link in pg_links:
                    link = pg_link.get('href')
                    link = urljoin(url,link)
                    image_page_link.append(link)
            
        except Exception as e:
            print(f'There is problem in getting image page link for {url} of page {i}')
    
    return image_page_link,len(image_page_link)

def get_image_and_details():
    image_page_link_list,_ = get_page_urls("http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/")
    count = 0
    os.makedirs("scraped_images", exist_ok=True)
    image_names_with_height = {}
    for img_links in image_page_link_list:
        count=count+1
        print(f'We are on {count} image')
        try:
            response = requests.get(img_links)
        except Exception as e:
            print(f'There is problem while accesing the given url: {img_links} and error is: {e} on image number {count}')
        
        try:
            soup = BeautifulSoup(response.content,'html.parser')
            parsing_2 = soup.find('div',{'class':'content_body'}).find('div').find('div',{'class':'eMessage'}).find_all('div')[0]
            parsing_3 = soup.find('div',{'class':'content_body'}).find('span')
        except Exception as e:
            print(f'There is problem in parsing the link {img_links}')

        try:
            img_tags = parsing_2.find_all('img')
            text = parsing_3.text
            height = text.split(" ")[2]
            img_url = img_tags[0].get('src')
            img_url = img_url.strip()
            if img_url:
                     img_url = urljoin("http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai",img_url)
                     img_name = os.path.basename(img_url)
                     img_path = os.path.join("scraped_images", img_name)
                     try:
                        urlretrieve(img_url, img_path)
                        print(f"Downloaded: {img_url}")
                        image_names_with_height[img_name] = height
                        print(f"{img_name}:{height}")
                     except Exception as e:
                        print(f"Failed to download: {img_url}")
                        print(f"Error: {e}")
        except Exception as e:
            print("There is problem in getting imag_tags and text")
        
        
    
    return image_names_with_height

def getting_dataframe():
    image_names_with_height = get_image_and_details()
    img_name = []
    height = []
    for key,value in image_names_with_height.items():
        img_name.append(key)
        height.append(value)
    
    data_1 = pd.DataFrame({"image_name":img_name,"height":height})
    data_1.to_csv("image_names_with_height_data.csv")
    
    return data_1

In [ ]:
getting_dataframe()